# Importation des modules

In [40]:
# Pour manipuler et visualiser les données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pour séparer et évaluer les données
from sklearn.model_selection import train_test_split, cross_validate, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

# Pour préparer les données
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# Pour créer des arbres de classification
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

# Pour utiliser les métriques
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Pour exporter notre modèle
import pickle

# Création du dataset et Dummy Classifier

On commence par créer un dataset avec les variables :
<p style='color: #FFA07A'> NAICS, NoEmp, NewExist, FranchiseCode,  UrbanRural, CreateJob, RetainedJob, MIS_Status </p>

In [41]:
df = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_8283/3668100114.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/SBAnational_clean.csv")


In [42]:
# Dataframe contenant les variables explicatives
X = df[['NAICS', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural']]

# On transforme les valeurs de MIS_Status en 0 ou 1
y = df['MIS_Status'].astype('category').cat.codes

On crée maintenant nos jeu de données d'entraînement et de test :

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42, stratify=y)

On sépare les variables numériques et catégorielle :

In [44]:
var_num = ['NAICS', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural']

Puis on crée un transformateur de colonne :

In [45]:
preprocessor = make_column_transformer(
    (StandardScaler(), var_num)
)

On commence notre modélisation par un Dummy Classifier qui servira de point de comparaison :

In [46]:
dummy = make_pipeline(preprocessor, DummyClassifier())

dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

recall = recall_score(y_test, y_pred)
print("Recall: ", recall)

f1 = f1_score(y_test, y_pred)
print("F1-score: ", f1)

Accuracy:  0.8237819517017371
Precision:  0.8237819517017371
Recall:  1.0
F1-score:  0.9033776772854687


# Random Forest Classifier

Nous allons maintenant utiliser un Random Forest Classififier sans paramètres pour le comparer avec notre Dummy Classifier :

In [47]:
# Entraînement du modèle
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier())
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NAICS', 'NoEmp', 'NewExist',
                                                   'CreateJob', 'RetainedJob',
                                                   'UrbanRural'])])),
                ('randomforestclassifier', RandomForestClassifier())])

On affiche l'importance des features :

In [48]:
importances = rf_pipe[-1].feature_importances_
features = pd.DataFrame({"feature": X_train.columns, "importance": importances})
features

,feature,importance
0,NAICS,0.629164
1,NoEmp,0.100304
2,NewExist,0.010589
3,CreateJob,0.082278
4,RetainedJob,0.116228
5,UrbanRural,0.061437


In [49]:
y_pred = rf_pipe.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

recall = recall_score(y_test, y_pred)
print("Recall: ", recall)

f1 = f1_score(y_test, y_pred)
print("F1-score: ", f1)

Accuracy:  0.8051800593136562
Precision:  0.8379981543094446
Recall:  0.9464787766577522
F1-score:  0.8889411279778964


En utilisant un Random Forest Classifier sans paramètres, on obtient de bons résultats, proches de notre Dummy Classifier. <br>
On va essayer d'affiner notre modèle avec une cross-validation puis en cherchant les meilleurs hyperparamètres :

In [50]:
scoring = {'recall': 'recall', 'f1': 'f1', 'precision': 'precision', 'accuracy': 'accuracy'}
cross_val = cross_validate(rf_pipe, X_train, y_train, cv=5, scoring=scoring)

In [51]:
# On affiche les différentes métriques avec leur écart-type
print("Accuracy: %0.3f (+/- %0.3f)" % (cross_val['test_accuracy'].mean(), cross_val['test_accuracy'].std() * 2))
print("Precision: %0.3f (+/- %0.3f)" % (cross_val['test_precision'].mean(), cross_val['test_precision'].std() * 2))
print("Recall: %0.3f (+/- %0.3f)" % (cross_val['test_recall'].mean(), cross_val['test_recall'].std() * 2))
print("F1: %0.3f (+/- %0.3f)" % (cross_val['test_f1'].mean(), cross_val['test_f1'].std() * 2))

Accuracy: 0.802 (+/- 0.001)
Precision: 0.838 (+/- 0.001)
Recall: 0.943 (+/- 0.001)
F1: 0.887 (+/- 0.001)


Les résultats obtenus sont quasiment les mêmes que ceux obtenus sans cross validation. On recherche maintenant des hyperparamètres susceptibles d'améliorer le modèle :

In [52]:
# On définit les paramètres à rechercher avec un intervalle de recherche
param_dist = {'n_estimators': np.arange(10, 100), 
              'max_depth': np.arange(1, 11),
              'min_samples_split': np.arange(2, 10),
              'min_samples_leaf': np.arange(1, 10),
              'criterion': ['gini', 'entropy']}

# On crée un objet Random Search 
random_search = RandomizedSearchCV(rf_pipe, param_distributions=param_dist,
                                   n_iter=50, cv=5, n_jobs=-1)

# On entrâine le modèle
random_search.fit(X_train, y_train)

# On récupère les meilleurs hyperparamètres
best_params = random_search.best_params_
print(best_params)

ValueError: Invalid parameter 'n_estimators' for estimator Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NAICS', 'NoEmp', 'NewExist',
                                                   'CreateJob', 'RetainedJob',
                                                   'UrbanRural'])])),
                ('randomforestclassifier', RandomForestClassifier())]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
# # Get the best estimator from the random search
# best_estimator = random_search.best_estimator_

# # Use the best estimator to make predictions on the test set
# y_pred = best_estimator.predict(X_test)

# # Calculate precision, recall and AUC-ROC
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)

# # Print the results
# print("Precision: {:.2f}".format(precision))
# print("Recall: {:.2f}".format(recall))
# print("AUC-ROC: {:.2f}".format(auc))